In [1]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import json
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
from flask import Flask, render_template, request, jsonify

In [2]:

# API endpoint URL
url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key=H0bVyMLzqj1Wd45IS1IxBUygOOwx7nH50dOa68hC&fuel_type=ELEC"
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
response = session.get(url)
response.raise_for_status()


In [3]:

data = response.json()['fuel_stations']
df_stations = pd.DataFrame(data)[['latitude', 'longitude', 'street_address', 'state', 'city', 'zip', 'country', 'id', 'open_date', 'fuel_type_code']]
df_stations 


,latitude,longitude,street_address,state,city,zip,country,id,open_date,fuel_type_code
0,34.248319,-118.387971,11797 Truesdale St,CA,Sun Valley,91352,US,1517,1999-10-15,ELEC
1,34.040539,-118.271387,1201 S Figueroa St,CA,Los Angeles,90015,US,1523,1995-08-30,ELEC
2,34.059133,-118.248589,111 N Hope St,CA,Los Angeles,90012,US,1525,1999-10-15,ELEC
3,33.759802,-118.096665,6801 E 2nd St,CA,Long Beach,90803,US,1531,2018-05-01,ELEC
4,33.770508,-118.265628,161 N Island Ave,CA,Wilmington,90744,US,1552,1999-10-15,ELEC
...,...,...,...,...,...,...,...,...,...,...
71651,36.637168,-120.625984,46365 West Panoche Road,CA,Firebaugh,93622,US,332488,2024-04-17,ELEC
71652,36.637262,-120.626035,46365 West Panoche Road,CA,Firebaugh,93622,US,332489,2024-04-17,ELEC
71653,36.637292,-120.626070,46365 West Panoche Road,CA,Firebaugh,93622,US,332490,2024-04-17,ELEC
71654,36.637313,-120.626080,46365 West Panoche Road,CA,Firebaugh,93622,US,332491,2024-04-17,ELEC


In [4]:
import os
current_directory = os.getcwd()

print("Current Directory:", current_directory)

Current Directory: c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current


In [5]:
df_vt_reg_pre = pd.read_csv(r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\VT_EV_Registrations_Zip.csv")
df_vt_reg_pre

,State,ZIP Code,Registration Date,Vehicle Make,Vehicle Model,Vehicle Model Year,Drivetrain Type,Vehicle GVWR Class,Vehicle Category,Vehicle Count,DMV Snapshot ID,DMV Snapshot (Date),Latest DMV Snapshot Flag
0,VT,05472,11/1/2017,CHEVROLET,VOLT,2017,PHEV,1,Light-Duty (Class 1-2A),1,1,DMV Direct (12/31/2018),False
1,VT,05065,5/1/2018,CHEVROLET,VOLT,2017,PHEV,1,Light-Duty (Class 1-2A),1,1,DMV Direct (12/31/2018),False
2,VT,05143,6/1/2017,CHEVROLET,VOLT,2017,PHEV,1,Light-Duty (Class 1-2A),1,1,DMV Direct (12/31/2018),False
3,VT,05855,4/1/2018,CHEVROLET,VOLT,2017,PHEV,1,Light-Duty (Class 1-2A),1,1,DMV Direct (12/31/2018),False
4,VT,05255,2/1/2017,CHEVROLET,VOLT,2017,PHEV,1,Light-Duty (Class 1-2A),1,1,DMV Direct (12/31/2018),False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51387,VT,05672,3/1/2023,JEEP,WRANGLER,2023,PHEV,2A,Light-Duty (Class 1-2A),3,10,DMV Snapshot (3/23/2023),False
51388,VT,05452,6/1/2023,JEEP,WRANGLER,2023,PHEV,2A,Light-Duty (Class 1-2A),4,11,DMV Snapshot (6/24/2023),False
51389,VT,37923,10/1/2023,JEEP,WRANGLER,2023,PHEV,2A,Light-Duty (Class 1-2A),3,12,DMV Snapshot (10/2/2023),True
51390,VT,05403,10/1/2023,JEEP,WRANGLER,2023,PHEV,2A,Light-Duty (Class 1-2A),3,12,DMV Snapshot (10/2/2023),True


In [6]:
df_stations['open_date'] = pd.to_datetime(df_stations['open_date'])
vt_stations_sorted = df_stations[(df_stations['state'] == 'VT')]
vt_stations_pre = vt_stations_sorted.sort_values(by='open_date')
vt_station = vt_stations_pre.dropna(subset=['open_date'])
vt_station

,latitude,longitude,street_address,state,city,zip,country,id,open_date,fuel_type_code
91,44.469281,-73.154972,1200 Airport Dr,VT,South Burlington,05403,US,32973,2004-01-15,ELEC
36961,44.445929,-73.113631,78 Marshall Avenue,VT,Williston,5495,US,195422,2011-09-28,ELEC
2017,44.426201,-73.211998,1675 Shelburne Rd,VT,South Burlington,05403,US,49408,2011-10-01,ELEC
36699,43.176820,-73.057215,4869 Main Street,VT,Manchester,5255,US,195003,2012-01-10,ELEC
660,44.015999,-73.165412,14 Seminary St,VT,Middlebury,05753,US,44012,2012-03-01,ELEC
...,...,...,...,...,...,...,...,...,...,...
69146,44.430280,-72.017250,8 Memorial Drive,VT,Saint Johnsbury,05819,US,327997,2024-02-09,ELEC
69517,44.416989,-72.015416,51 Depot St,VT,ST. Johnsbury,5819,US,328894,2024-02-21,ELEC
70444,44.951065,-72.157436,4441 U.S. 5,VT,Derby,05829,US,330797,2024-03-14,ELEC
70306,44.437875,-72.015730,220 Memorial Drive,VT,Saint Johnsbury,05819,US,330588,2024-03-14,ELEC


In [7]:
vt_station_dropped_zips = vt_station.drop_duplicates(subset='zip')
vt_station_dropped_zips

,latitude,longitude,street_address,state,city,zip,country,id,open_date,fuel_type_code
91,44.469281,-73.154972,1200 Airport Dr,VT,South Burlington,05403,US,32973,2004-01-15,ELEC
36961,44.445929,-73.113631,78 Marshall Avenue,VT,Williston,5495,US,195422,2011-09-28,ELEC
36699,43.176820,-73.057215,4869 Main Street,VT,Manchester,5255,US,195003,2012-01-10,ELEC
660,44.015999,-73.165412,14 Seminary St,VT,Middlebury,05753,US,44012,2012-03-01,ELEC
2015,44.478000,-73.210380,82 S Winooski Ave,VT,Burlington,05401,US,49404,2012-05-01,ELEC
...,...,...,...,...,...,...,...,...,...,...
58825,43.519166,-73.236184,168 Church St,VT,Poultney,05764,US,262693,2023-06-14,ELEC
59969,43.876556,-72.807165,162 North Main St,VT,Rochester,05767,US,302262,2023-07-08,ELEC
64100,44.414806,-72.132179,421 US-2,VT,Danville,05828,US,312506,2023-10-06,ELEC
69517,44.416989,-72.015416,51 Depot St,VT,ST. Johnsbury,5819,US,328894,2024-02-21,ELEC


In [8]:
df_stations['open_date'] = pd.to_datetime(df_stations['open_date'])
ca_stations_sorted = df_stations[(df_stations['state'] == 'CA')]
ca_stations_pre = ca_stations_sorted.sort_values(by='open_date')
ca_station = ca_stations_pre.dropna(subset=['open_date'])
ca_station

,latitude,longitude,street_address,state,city,zip,country,id,open_date,fuel_type_code
1,34.040539,-118.271387,1201 S Figueroa St,CA,Los Angeles,90015,US,1523,1995-08-30,ELEC
8,34.068720,-118.064000,9530 Telstar Ave,CA,El Monte,91731,US,1583,1996-10-15,ELEC
10,32.899470,-117.243000,10666 N Torrey Pines Rd,CA,La Jolla,92037,US,6355,1997-07-30,ELEC
12,33.909914,-117.459053,1299 Galleria at Tyler,CA,Riverside,92503,US,6425,1997-08-30,ELEC
15,34.145119,-118.150133,33 E Green St,CA,Pasadena,91105,US,6507,1997-08-30,ELEC
...,...,...,...,...,...,...,...,...,...,...
71643,33.946263,-118.145342,7550 Firestone Boulevard,CA,Downey,90241,US,332475,2024-04-17,ELEC
71625,33.842734,-118.330389,485 Crenshaw Blvd,CA,Torrance,90503,US,332455,2024-04-17,ELEC
71654,36.637313,-120.626080,46365 West Panoche Road,CA,Firebaugh,93622,US,332491,2024-04-17,ELEC
71648,36.637481,-120.626448,46365 West Panoche Road,CA,Firebaugh,93622,US,332485,2024-04-17,ELEC


In [9]:
file_path = r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\ca_station.csv"
ca_station.to_csv(file_path, index=False)

In [10]:
df_stations['open_date'] = pd.to_datetime(df_stations['open_date'])
tn_stations_sorted = df_stations[(df_stations['state'] == 'TN')]
tn_stations_pre = tn_stations_sorted.sort_values(by='open_date')
tn_station = tn_stations_pre.dropna(subset=['open_date'])
tn_station

,latitude,longitude,street_address,state,city,zip,country,id,open_date,fuel_type_code
1996,35.922002,-84.142112,942 Corridor Park Blvd,TN,Knoxville,37932,US,49206,2010-01-20,ELEC
7484,35.866934,-86.449001,3050 Medical Center Pkwy,TN,Murfreesboro,37129,US,104604,2011-01-01,ELEC
213,36.010279,-84.268049,101 S Illinois Ave,TN,Oak Ridge,37830,US,39705,2011-03-15,ELEC
1721,36.190920,-86.793340,25 Vantage Way,TN,Nashville,37228,US,47843,2011-03-15,ELEC
1720,36.110330,-86.743730,307 Thompson Ln,TN,Nashville,37211,US,47842,2011-03-15,ELEC
...,...,...,...,...,...,...,...,...,...,...
71082,36.413591,-89.049201,705 E Reelfoot Ave,TN,Union City,38261,US,331829,2024-04-04,ELEC
71541,36.035720,-83.444770,134 Sharon Drive,TN,Dandridge,37725,US,332369,2024-04-16,ELEC
71502,35.021544,-89.723412,4651 South Houston Levee Road,TN,Collierville,38017,US,332321,2024-04-16,ELEC
71511,36.456579,-81.843664,1203 Harbin Hill Rd,TN,Mountain City,37683,US,332331,2024-04-16,ELEC


In [11]:
file_path = r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\tn_station.csv"
tn_station.to_csv(file_path, index=False)

In [12]:
df_stations['open_date'] = pd.to_datetime(df_stations['open_date'])
nc_stations_sorted = df_stations[(df_stations['state'] == 'NC')]
nc_stations_pre = nc_stations_sorted.sort_values(by='open_date')
nc_station = nc_stations_pre.dropna(subset=['open_date'])
nc_station

,latitude,longitude,street_address,state,city,zip,country,id,open_date,fuel_type_code
20899,34.789404,-77.402415,1408 Western Blvd,NC,Jacksonville,28546,US,166781,2010-01-01,ELEC
203,35.778416,-78.643470,285 W Hargett St,NC,Raleigh,27601,US,39016,2010-10-15,ELEC
204,35.774350,-78.642287,215 W Cabarrus St,NC,Raleigh,27601,US,39017,2011-02-15,ELEC
1350,35.937840,-79.996290,1810 S Main St,NC,High Point,27260,US,47012,2011-03-15,ELEC
471,34.634487,-78.997134,1949 Roberts Ave,NC,Lumberton,28358,US,40069,2011-03-15,ELEC
...,...,...,...,...,...,...,...,...,...,...
71416,34.902919,-79.711290,1042 W Hamlet Ave,NC,Hamlet,28345,US,332229,2024-04-13,ELEC
71415,34.717752,-76.660333,214 Pollock St.,NC,Beaufort,28516,US,332228,2024-04-13,ELEC
71423,35.209522,-80.921550,3610 Nobles Ave.,NC,Charlotte,28208,US,332236,2024-04-13,ELEC
71521,35.226320,-80.849110,415 South Mint Street,NC,Charlotte,28202,US,332348,2024-04-16,ELEC


In [13]:
file_path = r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\nc_station.csv"
nc_station.to_csv(file_path, index=False)

In [14]:
ca_reg_lat_long = pd.read_csv(r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\CA_EV_Registration_LatLong_2020-2022.csv")
ca_reg_lat_long

,State,Zip Code,Date,Fuel,Vehicle Count,Latitude,Longitude
0,CA,92220,1/1/2020,Battery Electric,1,33.931748,-116.945951
1,CA,92220,1/1/2020,Battery Electric,1,33.924926,-116.899774
2,CA,92220,1/1/2020,Battery Electric,1,33.911467,-116.868371
3,CA,92220,1/1/2020,Battery Electric,1,33.928077,-116.945267
4,CA,92220,1/1/2020,Battery Electric,1,33.929657,-116.946246
...,...,...,...,...,...,...,...
614066,CA,96091,1/1/2022,Battery Electric,1,41.084381,-122.718455
614067,CA,96101,1/1/2022,Battery Electric,1,41.482442,-120.542075
614068,CA,96101,1/1/2022,Battery Electric,1,41.495392,-120.559776
614069,CA,96101,1/1/2022,Battery Electric,1,41.482442,-120.542075


In [15]:
tn_reg_lat_long = pd.read_csv(r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\TN_EV_Registration_LatLong_2020-2022.csv")
tn_reg_lat_long

,State,Zip Code,Date,Fuel,Vehicle Count,Latitude,Longitude
0,TN,47189,3/1/2022,BEV,1,36.1547,-86.2973
1,TN,47189,3/1/2023,BEV,1,36.1547,-86.2973
2,TN,47189,12/1/2022,BEV,1,36.1547,-86.2973
3,TN,47189,6/1/2023,BEV,1,36.1547,-86.2973
4,TN,47189,12/1/2021,BEV,1,36.1547,-86.2973
...,...,...,...,...,...,...,...
51772,TN,47095,9/1/2023,BEV,1,36.3352,-89.4935
51773,TN,47095,12/1/2023,BEV,1,36.3352,-89.4935
51774,TN,47095,12/1/2023,BEV,1,36.3352,-89.4935
51775,TN,47095,9/1/2023,BEV,1,36.3352,-89.4935


In [16]:
df_vt_reg = df_vt_reg_pre[df_vt_reg_pre['Drivetrain Type'] == 'BEV']
df_vt_reg.rename(columns={'ZIP Code': 'zip'}, inplace=True)
df_vt_reg

C:\Users\Albrecht\AppData\Local\Temp\ipykernel_3388\619163181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vt_reg.rename(columns={'ZIP Code': 'zip'}, inplace=True)


,State,zip,Registration Date,Vehicle Make,Vehicle Model,Vehicle Model Year,Drivetrain Type,Vehicle GVWR Class,Vehicle Category,Vehicle Count,DMV Snapshot ID,DMV Snapshot (Date),Latest DMV Snapshot Flag
3172,VT,05660,10/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,12,DMV Snapshot (10/2/2023),True
3173,VT,05819,10/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,12,DMV Snapshot (10/2/2023),True
3174,VT,05701,1/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,9,DMV Snapshot (1/10/2023),False
3175,VT,05148,10/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,12,DMV Snapshot (10/2/2023),True
3176,VT,05046,3/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,10,DMV Snapshot (3/23/2023),False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51304,VT,05482,10/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,12,DMV Snapshot (10/2/2023),True
51305,VT,05660,6/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,11,DMV Snapshot (6/24/2023),False
51306,VT,05251,3/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,10,DMV Snapshot (3/23/2023),False
51307,VT,05445,6/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,11,DMV Snapshot (6/24/2023),False


In [17]:
file_path = r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\df_vt_reg.csv"
df_vt_reg.to_csv(file_path, index=False)

In [18]:
vt_station['zip'] = vt_station['zip'].astype(str).str.zfill(5)
file_path = r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\vt_station.csv"
vt_station.to_csv(file_path, index=False)

C:\Users\Albrecht\AppData\Local\Temp\ipykernel_3388\3726313671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vt_station['zip'] = vt_station['zip'].astype(str).str.zfill(5)


In [20]:
df_vt_reg['zip'] = df_vt_reg['zip'].astype(str)
vt_station_dropped_zips['zip'] = vt_station_dropped_zips['zip'].astype(str)

df_vt_reg_lat_long = pd.merge(df_vt_reg, vt_station_dropped_zips[['zip', 'latitude', 'longitude']], on='zip', how='left')


df_vt_reg_lat_long['latitude'].fillna(df_vt_reg_lat_long['latitude'].mean(), inplace=True)
df_vt_reg_lat_long['longitude'].fillna(df_vt_reg_lat_long['longitude'].mean(), inplace=True)

df_vt_reg_lat_long


C:\Users\Albrecht\AppData\Local\Temp\ipykernel_3388\17643055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vt_reg['zip'] = df_vt_reg['zip'].astype(str)
C:\Users\Albrecht\AppData\Local\Temp\ipykernel_3388\17643055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vt_station_dropped_zips['zip'] = vt_station_dropped_zips['zip'].astype(str)


,State,zip,Registration Date,Vehicle Make,Vehicle Model,Vehicle Model Year,Drivetrain Type,Vehicle GVWR Class,Vehicle Category,Vehicle Count,DMV Snapshot ID,DMV Snapshot (Date),Latest DMV Snapshot Flag,latitude,longitude
0,VT,05660,10/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,12,DMV Snapshot (10/2/2023),True,44.089596,-72.896393
1,VT,05819,10/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,12,DMV Snapshot (10/2/2023),True,44.445764,-72.009698
2,VT,05701,1/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,9,DMV Snapshot (1/10/2023),False,43.573814,-72.966623
3,VT,05148,10/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,12,DMV Snapshot (10/2/2023),True,44.089596,-72.896393
4,VT,05046,3/1/2023,CHEVROLET,BOLT EV,2023,BEV,1,Light-Duty (Class 1-2A),1,10,DMV Snapshot (3/23/2023),False,44.089596,-72.896393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25949,VT,05482,10/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,12,DMV Snapshot (10/2/2023),True,44.408287,-73.218887
25950,VT,05660,6/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,11,DMV Snapshot (6/24/2023),False,44.089596,-72.896393
25951,VT,05251,3/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,10,DMV Snapshot (3/23/2023),False,43.233306,-73.081222
25952,VT,05445,6/1/2023,RIVIAN,R1T,2022,BEV,2A,Light-Duty (Class 1-2A),3,11,DMV Snapshot (6/24/2023),False,44.310190,-73.249800


In [21]:
desired_columns = ['State', 'zip', 'Registration Date', 'Drivetrain Type', 'Vehicle Count', 'latitude', 'longitude']
vt_reg_lat_long = df_vt_reg_lat_long.loc[:, desired_columns]
vt_reg_lat_long


,State,zip,Registration Date,Drivetrain Type,Vehicle Count,latitude,longitude
0,VT,05660,10/1/2023,BEV,1,44.089596,-72.896393
1,VT,05819,10/1/2023,BEV,1,44.445764,-72.009698
2,VT,05701,1/1/2023,BEV,1,43.573814,-72.966623
3,VT,05148,10/1/2023,BEV,1,44.089596,-72.896393
4,VT,05046,3/1/2023,BEV,1,44.089596,-72.896393
...,...,...,...,...,...,...,...
25949,VT,05482,10/1/2023,BEV,3,44.408287,-73.218887
25950,VT,05660,6/1/2023,BEV,3,44.089596,-72.896393
25951,VT,05251,3/1/2023,BEV,3,43.233306,-73.081222
25952,VT,05445,6/1/2023,BEV,3,44.310190,-73.249800


In [22]:
vt_reg_lat_long = vt_reg_lat_long.rename(columns={
    'Registration Date': 'Date',
    'zip': 'Zip Code',
    'Drivetrain Type': 'Fuel',
    'latitude': 'Latitude',
    'longitude': 'Longitude'
})

vt_reg_lat_long


,State,Zip Code,Date,Fuel,Vehicle Count,Latitude,Longitude
0,VT,05660,10/1/2023,BEV,1,44.089596,-72.896393
1,VT,05819,10/1/2023,BEV,1,44.445764,-72.009698
2,VT,05701,1/1/2023,BEV,1,43.573814,-72.966623
3,VT,05148,10/1/2023,BEV,1,44.089596,-72.896393
4,VT,05046,3/1/2023,BEV,1,44.089596,-72.896393
...,...,...,...,...,...,...,...
25949,VT,05482,10/1/2023,BEV,3,44.408287,-73.218887
25950,VT,05660,6/1/2023,BEV,3,44.089596,-72.896393
25951,VT,05251,3/1/2023,BEV,3,43.233306,-73.081222
25952,VT,05445,6/1/2023,BEV,3,44.310190,-73.249800


In [23]:
file_path = r"c:\Users\Albrecht\Desktop\BC Class Repo\git_hub\project3-alternating-current\vt_reg_lat_long.csv"
vt_reg_lat_long.to_csv(file_path, index=False)

In [24]:
import plotly.express as px

# Marker map
fig_marker = px.scatter_mapbox(
    vt_station,
    lat="latitude",
    lon="longitude",
    hover_name="street_address",
    zoom=6.5,
    height=600
)
fig_marker.update_layout(
    mapbox_style="carto-positron",
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig_marker.show()


In [25]:
# Heatmap
fig_heatmap = px.density_mapbox(vt_reg_lat_long, lat="Latitude", lon="Longitude", 
                                 radius=3, zoom=6.5, height=600)
fig_heatmap.update_layout(mapbox_style="carto-positron")
fig_heatmap.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_heatmap.show()



In [41]:
from flask import Flask, render_template, request
import plotly.io as pio
import plotly.express as px
import pandas as pd
import os

app = Flask(__name__)

# Dataframes from CSV files
vt_station = pd.read_csv('vt_station.csv')
vt_reg_lat_long = pd.read_csv('vt_reg_lat_long.csv')

# Define the years list
years = ['2019', '2020', '2021', '2022']

# Create a static directory
static_dir = os.path.join(app.root_path, 'static')
if not os.path.exists(static_dir):
    os.makedirs(static_dir)

# Create visualizations and save HTML files for each year
for year in years:
    combined_years = [str(y) for y in range(2019, int(year) + 1)]  # Combine years up to the selected year
    vt_reg_lat_long_year = vt_reg_lat_long[vt_reg_lat_long['Date'].str[-4:].isin(combined_years)]

    # Create the heatmap figure
    fig_heatmap = px.density_mapbox(vt_reg_lat_long_year, lat="Latitude", lon="Longitude",
                                     radius=3, zoom=6.5, height=600)
    fig_heatmap.update_layout(mapbox_style="carto-positron")
    fig_heatmap.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    # Save the heatmap figure as an HTML file in the static directory
    heatmap_html_path = os.path.join(static_dir, f'heatmap_{year}.html')
    pio.write_html(fig_heatmap, file=heatmap_html_path, auto_open=False)

    # Filter data for the specific year for the marker map
    vt_station_year = vt_station[vt_station['open_date'].apply(lambda x: int(x.split('-')[0])) <= int(year)]

    # Create the marker map figure
    fig_marker = px.scatter_mapbox(vt_station_year, lat="latitude", lon="longitude",
                                   hover_name="street_address", zoom=6.5, height=600)
    fig_marker.update_layout(mapbox_style="carto-positron")
    fig_marker.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    # Save the marker map figure as an HTML file in the static directory
    marker_map_html_path = os.path.join(static_dir, f'marker_map_{year}.html')
    pio.write_html(fig_marker, file=marker_map_html_path, auto_open=False)

#Define routes:
@app.route('/')
def index():
    return render_template('index_vt.html', years=years)

@app.route('/heatmap/<year>')
def heatmap(year):
    heatmap_html_path = os.path.join(static_dir, f'heatmap_{year}.html')
    with open(heatmap_html_path, 'r') as file:
        heatmap_html = file.read()
    return heatmap_html

@app.route('/marker_map/<year>')
def marker_map(year):
    marker_map_html_path = os.path.join(static_dir, f'marker_map_{year}.html')
    with open(marker_map_html_path, 'r') as file:
        marker_map_html = file.read()
    return marker_map_html

if __name__ == '__main__':
    app.run()




 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Apr/2024 21:18:04] "GET / HTTP/1.1" 200 -


In [38]:
from flask import Flask, render_template
import plotly.io as pio
import plotly.express as px
import pandas as pd
import os

app = Flask(__name__)

# Dataframes from CSV files
ca_station = pd.read_csv('ca_station.csv')
ca_reg_lat_long = pd.read_csv('CA_EV_Registration_latLong_2020-2022.csv')

# Define the years list
years = ['2020', '2021', '2022']

# Create a static directory if it doesn't exist
static_dir = os.path.join(app.root_path, 'static')
if not os.path.exists(static_dir):
    os.makedirs(static_dir)

# Create visualizations and save HTML files for each year
for year in years:
    combined_years = [str(y) for y in range(2020, int(year) + 1)]  # Combine years up to the selected year for CA data
    ca_reg_lat_long_year = ca_reg_lat_long[ca_reg_lat_long['Date'].str[-4:].isin(combined_years)]

    # Create the heatmap figure
    fig_heatmap = px.density_mapbox(ca_reg_lat_long_year, lat="Latitude", lon="Longitude",
                                     radius=3, zoom=5, height=600, center={"lat": 36.7783, "lon": -119.4179})
    fig_heatmap.update_layout(mapbox_style="carto-positron")
    fig_heatmap.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Save the heatmap figure as an HTML file in the static directory
    heatmap_html_path = os.path.join(static_dir, f'heatmap_{year}.html')
    pio.write_html(fig_heatmap, file=heatmap_html_path, auto_open=False)

    # Filter data for the specific year for the marker map
    ca_station_year = ca_station[ca_station['open_date'].apply(lambda x: int(x.split('-')[0])) <= int(year)]

    # Create the marker map figure
    fig_marker = px.scatter_mapbox(ca_station_year, lat="latitude", lon="longitude",
                                   hover_name="street_address", zoom=5, height=600, center={"lat": 36.7783, "lon": -119.4179})
    fig_marker.update_layout(mapbox_style="carto-positron")
    fig_marker.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Save the marker map figure as an HTML file in the static directory
    marker_map_html_path = os.path.join(static_dir, f'marker_map_{year}.html')
    pio.write_html(fig_marker, file=marker_map_html_path, auto_open=False)

# Define routes
@app.route('/')
def index():
    return render_template('index_ca.html', years=years)

@app.route('/heatmap/<year>')
def heatmap(year):
    heatmap_html_path = os.path.join(static_dir, f'heatmap_{year}.html')
    with open(heatmap_html_path, 'r') as file:
        heatmap_html = file.read()
    return heatmap_html

@app.route('/marker_map/<year>')
def marker_map(year):
    marker_map_html_path = os.path.join(static_dir, f'marker_map_{year}.html')
    with open(marker_map_html_path, 'r') as file:
        marker_map_html = file.read()
    return marker_map_html

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Apr/2024 21:15:52] "GET / HTTP/1.1" 200 -


In [35]:
from flask import Flask, render_template
import plotly.io as pio
import plotly.express as px
import pandas as pd
import os

app = Flask(__name__)

# Dataframes from CSV files
tn_station = pd.read_csv('tn_station.csv')
tn_reg_lat_long = pd.read_csv('TN_EV_Registration_LatLong_2020-2022.csv')

# Define the years list
years = ['2020', '2021', '2022']

# Create a static directory if it doesn't exist
static_dir = os.path.join(app.root_path, 'static')
if not os.path.exists(static_dir):
    os.makedirs(static_dir)

# Create visualizations and save HTML files for each year
for year in years:
    combined_years = [str(y) for y in range(2020, int(year) + 1)]  # Combine years up to the selected year for TN data
    tn_reg_lat_long_year = tn_reg_lat_long[tn_reg_lat_long['Date'].str[-4:].isin(combined_years)]

    # Create the heatmap figure
    fig_heatmap = px.density_mapbox(tn_reg_lat_long_year, lat="Latitude", lon="Longitude",
                                     radius=3, zoom=6, height=600, center={"lat": 35.5175, "lon": -86.5804})
    fig_heatmap.update_layout(mapbox_style="carto-positron")
    fig_heatmap.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Save the heatmap figure as an HTML file in the static directory
    heatmap_html_path = os.path.join(static_dir, f'heatmap_tn_{year}.html')
    pio.write_html(fig_heatmap, file=heatmap_html_path, auto_open=False)

    # Filter data for the specific year for the marker map
    tn_station_year = tn_station[tn_station['open_date'].apply(lambda x: int(x.split('-')[0])) <= int(year)]

    # Create the marker map figure
    fig_marker = px.scatter_mapbox(tn_station_year, lat="latitude", lon="longitude",
                                   hover_name="street_address", zoom=6, height=600, center={"lat": 35.5175, "lon": -86.5804})
    fig_marker.update_layout(mapbox_style="carto-positron")
    fig_marker.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Save the marker map figure as an HTML file in the static directory
    marker_map_html_path = os.path.join(static_dir, f'marker_map_tn_{year}.html')
    pio.write_html(fig_marker, file=marker_map_html_path, auto_open=False)

# Define routes:
    
@app.route('/')
def index():
    return render_template('index_tn.html', years=years)

@app.route('/heatmap/<year>')
def heatmap(year):
    heatmap_html_path = os.path.join(static_dir, f'heatmap_tn_{year}.html')
    with open(heatmap_html_path, 'r') as file:
        heatmap_html = file.read()
    return heatmap_html

@app.route('/marker_map/<year>')
def marker_map(year):
    marker_map_html_path = os.path.join(static_dir, f'marker_map_tn_{year}.html')
    with open(marker_map_html_path, 'r') as file:
        marker_map_html = file.read()
    return marker_map_html

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Apr/2024 21:11:22] "GET / HTTP/1.1" 200 -


In [36]:
from flask import Flask, render_template
import plotly.io as pio
import plotly.express as px
import pandas as pd
import os

app = Flask(__name__)

# Dataframes from CSV files
nc_station = pd.read_csv('nc_station.csv')
nc_reg_lat_long = pd.read_csv('NC_EV_Registration_LatLong_2020-2022.csv')

# Define the years list
years = ['2020', '2021', '2022']

# Create a static directory if it doesn't exist
static_dir = os.path.join(app.root_path, 'static')
if not os.path.exists(static_dir):
    os.makedirs(static_dir)

# Create visualizations and save HTML files for each year
for year in years:
    combined_years = [str(y) for y in range(2020, int(year) + 1)]  # Combine years up to the selected year for NC data
    nc_reg_lat_long_year = nc_reg_lat_long[nc_reg_lat_long['Date'].str[-4:].isin(combined_years)]

    # Create the heatmap figure
    fig_heatmap = px.density_mapbox(nc_reg_lat_long_year, lat="Latitude", lon="Longitude",
                                     radius=3, zoom=6, height=600, center={"lat": 35.7596, "lon": -79.0193})
    fig_heatmap.update_layout(mapbox_style="carto-positron")
    fig_heatmap.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Save the heatmap figure as an HTML file in the static directory
    heatmap_html_path = os.path.join(static_dir, f'heatmap_nc_{year}.html')
    pio.write_html(fig_heatmap, file=heatmap_html_path, auto_open=False)

    # Filter data for the specific year for the marker map
    nc_station_year = nc_station[nc_station['open_date'].apply(lambda x: int(x.split('-')[0])) <= int(year)]

    # Create the marker map figure
    fig_marker = px.scatter_mapbox(nc_station_year, lat="latitude", lon="longitude",
                                   hover_name="street_address", zoom=6, height=600, center={"lat": 35.7596, "lon": -79.0193})
    fig_marker.update_layout(mapbox_style="carto-positron")
    fig_marker.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Save the marker map figure as an HTML file in the static directory
    marker_map_html_path = os.path.join(static_dir, f'marker_map_nc_{year}.html')
    pio.write_html(fig_marker, file=marker_map_html_path, auto_open=False)

# Define routes
    
@app.route('/')
def index():
    return render_template('index_nc.html', years=years)

@app.route('/heatmap/<year>')
def heatmap(year):
    heatmap_html_path = os.path.join(static_dir, f'heatmap_nc_{year}.html')
    with open(heatmap_html_path, 'r') as file:
        heatmap_html = file.read()
    return heatmap_html

@app.route('/marker_map/<year>')
def marker_map(year):
    marker_map_html_path = os.path.join(static_dir, f'marker_map_nc_{year}.html')
    with open(marker_map_html_path, 'r') as file:
        marker_map_html = file.read()
    return marker_map_html

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Apr/2024 21:12:56] "GET / HTTP/1.1" 200 -
